In [ ]:
# find group name and sizes
# pick a group

# get embeddings for that group
# find topics
# find user membership across topics

In [1]:
import json
import os

import bz2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utils

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
DATA_PATH = '/sciclone/data10/twford/reddit/reddit/comments/'

In [16]:
soi = 'movies'

metadata = ['id', 'name', 'created_utc', 'parent_id', 
            'subreddit', 'subreddit_id', 'author']

reader = utils.read_file(
    os.path.join(DATA_PATH, 'RC_2010-01.bz2'), 
    return_type='metadata',
    metadata=metadata, 
    chunk_size=10000
)

res = []

# k will keep track of line and should align with embeddings
k = 0
for chunk in reader:
    for entry in chunk:
        if entry['subreddit'] == soi:
            entry['idx'] = k
            res.append(entry)
        k += 1

In [17]:
df_res = pd.DataFrame(res)
df_res.head()

,id,name,created_utc,parent_id,subreddit,subreddit_id,author,idx
0,c0i12y9,t1_c0i12y9,1262304136,t3_akd1b,movies,t5_2qh3s,bbittner,69
1,c0i12yf,t1_c0i12yf,1262304145,t1_c0i0xg6,movies,t5_2qh3s,passingby,75
2,c0i136p,t1_c0i136p,1262304464,t3_akc13,movies,t5_2qh3s,andhelostthem,293
3,c0i13at,t1_c0i13at,1262304618,t1_c0i0y0u,movies,t5_2qh3s,Wrathwilde,411
4,c0i13mq,t1_c0i13mq,1262305018,t3_akd1b,movies,t5_2qh3s,rawbinm,752


In [23]:
embeddings = utils.load_embeddings(2010, 1)

# grab just within subreddit
idx = df_res['idx'].values
embeddings = embeddings[idx]

In [24]:
embeddings.shape

(13303, 384)